In [1]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, ConcatDataset, WeightedRandomSampler, Subset, DataLoader
import os
import torchxrayvision as xrv
import torchvision.transforms as transforms
from skimage.color import rgb2gray
from skimage.transform import resize
import pydicom
from torchxrayvision.datasets import XRayCenterCrop
import pandas as pd
import wandb
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix
import helpers, train_utils, classes
from collections import Counter
import torch
import torch.nn as nn

c:\Users\user-pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dicom_dir_1 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1'
metadata_1 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1/FileDatabaseWithRadiology.xlsx')
dicom_dir_2 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2'
metadata_2 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2/Database_Training-2024.08.28.xlsx')

d1 = classes.DICOMDataset1(dicom_dir=dicom_dir_1, metadata_df=metadata_1, target_size=224) 
d2 = classes.DICOMDataset2(dicom_dir=dicom_dir_2, metadata_df=metadata_2, target_size=224)

# Split datasets and store indices
train_indices_d1, val_indices_d1, test_indices_d1 = helpers.split_dataset(d1)
train_indices_d2, val_indices_d2, test_indices_d2 = helpers.split_dataset(d2)

# Save indices for later use
split_indices = {
    'd1': {'train': train_indices_d1, 'val': val_indices_d1, 'test': test_indices_d1},
    'd2': {'train': train_indices_d2, 'val': val_indices_d2, 'test': test_indices_d2}
}

label = 'Profusion'
d1.set_target(target_label=label, target_size=224)
d2.set_target(target_label=label, target_size=224)

train_d1 = Subset(d1, train_indices_d1)
val_d1 = Subset(d1, val_indices_d1)
test_d1 = Subset(d1, test_indices_d1)

train_d2 = Subset(d2, train_indices_d2)
val_d2 = Subset(d2, val_indices_d2)
test_d2 = Subset(d2, test_indices_d2)

In [3]:
print(f'Length of D1: {len(d1)}')
print(print(f'Length of D2: {len(d2)}'), "\n")

print(d1.metadata_df['TBA-TBU Label'].value_counts(), "\n")
print(d1.metadata_df['Profusion Label'].value_counts(), "\n")
print(d1.metadata_df['Profusion and TBA-TBU Label'].value_counts(), "\n")
print(d1.metadata_df['Profusion or TBA-TBU Label'].value_counts(), "\n")


Length of D1: 1178
Length of D2: 857
None 

1.0    830
0.0    348
Name: TBA-TBU Label, dtype: int64 

1.0    814
0.0    364
Name: Profusion Label, dtype: int64 

1.0    693
0.0    485
Name: Profusion and TBA-TBU Label, dtype: int64 

1.0    951
0.0    227
Name: Profusion or TBA-TBU Label, dtype: int64 



In [4]:
d1_b = classes.DICOMDataset1_b(dicom_dir=dicom_dir_1, metadata_df=metadata_1, target_size=224)
d1_b.set_target(target_label=label, target_size=224)

print(d1_b.metadata_df['TBA-TBU Label'].value_counts(), "\n")
print(d1_b.metadata_df['Profusion Label'].value_counts(), "\n")
print(d1_b.metadata_df['Profusion and TBA-TBU Label'].value_counts(), "\n")
print(d1_b.metadata_df['Profusion or TBA-TBU Label'].value_counts(), "\n")



0.0    766
1.0    412
Name: TBA-TBU Label, dtype: int64 

1.0    814
0.0    364
Name: Profusion Label, dtype: int64 

0.0    848
1.0    330
Name: Profusion and TBA-TBU Label, dtype: int64 

1.0    896
0.0    282
Name: Profusion or TBA-TBU Label, dtype: int64 



In [9]:
# Find rows where either doctor is missing
missing_doctors_mask = (d1_b.metadata_df['strDoctor1'].isna()) & (d1_b.metadata_df['strDoctor2'].isna())

# Set all labels to 0 for these rows
d1_b.metadata_df.loc[missing_doctors_mask, 'TBA-TBU Label'] = 0
d1_b.metadata_df.loc[missing_doctors_mask, 'Profusion Label'] = 0
d1_b.metadata_df.loc[missing_doctors_mask, 'Profusion and TBA-TBU Label'] = 0
d1_b.metadata_df.loc[missing_doctors_mask, 'Profusion or TBA-TBU Label'] = 0

# Print updated label distributions
print("Updated label distributions:")
print("\nTBA-TBU Label:")
print(d1_b.metadata_df['TBA-TBU Label'].value_counts(), "\n")
print("Profusion Label:")
print(d1_b.metadata_df['Profusion Label'].value_counts(), "\n")
print("Profusion and TBA-TBU Label:")
print(d1_b.metadata_df['Profusion and TBA-TBU Label'].value_counts(), "\n")
print("Profusion or TBA-TBU Label:")
print(d1_b.metadata_df['Profusion or TBA-TBU Label'].value_counts(), "\n")

Updated label distributions:

TBA-TBU Label:
0.0    766
1.0    412
Name: TBA-TBU Label, dtype: int64 

Profusion Label:
1.0    814
0.0    364
Name: Profusion Label, dtype: int64 

Profusion and TBA-TBU Label:
0.0    848
1.0    330
Name: Profusion and TBA-TBU Label, dtype: int64 

Profusion or TBA-TBU Label:
1.0    896
0.0    282
Name: Profusion or TBA-TBU Label, dtype: int64 



In [8]:
# Check how many rows were actually filtered out
print("Original dataset size:", len(d1_b.metadata_df))
print("Filtered dataset size:", len(filtered_d1b))

# Check how many rows have both doctors missing
both_missing = d1_b.metadata_df[
    (d1_b.metadata_df['strDoctor1'].isna()) & 
    (d1_b.metadata_df['strDoctor2'].isna())
].shape[0]
print("Rows with both doctors missing:", both_missing)

# Check if the filtered rows had any positive labels
rows_removed = d1_b.metadata_df[
    (d1_b.metadata_df['strDoctor1'].isna()) & 
    (d1_b.metadata_df['strDoctor2'].isna())
]
print("\nLabel counts in removed rows:")
print("TBA-TBU Label:", rows_removed['TBA-TBU Label'].value_counts())
print("Profusion Label:", rows_removed['Profusion Label'].value_counts())

Original dataset size: 1178
Filtered dataset size: 964
Rows with both doctors missing: 214

Label counts in removed rows:
TBA-TBU Label: 0.0    214
Name: TBA-TBU Label, dtype: int64
Profusion Label: 0.0    214
Name: Profusion Label, dtype: int64


In [ ]:
def create_weighted_sampler(dataset, target_label):
    # Calculate class weights
    class_counts = np.bincount([label for _, label in dataset])
    class_weights = 1. / class_counts
    sample_weights = [class_weights[label] for _, label in dataset]

    # Create a weighted sampler
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
    return sampler

# Create the base datasets
train_d1 = Subset(d1, train_indices_d1)
train_d2 = Subset(d2, train_indices_d2)

# Define augmentations
augmentations_list = [
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
]

# Create augmented datasets
augmented_train_d1 = classes.AugmentedDataset(base_dataset=train_d1, augmentations_list=augmentations_list)
augmented_train_d2 = classes.AugmentedDataset(base_dataset=train_d2, augmentations_list=augmentations_list)

    # Create dataloaders
train_loader_d1, train_aug_loader_d1, val_loader_d1, test_loader_d1 = helpers.create_dataloaders(
    train_d1, augmented_train_d1, val_d1, test_d1, batch_size=32, target=label
)

train_loader_d2, train_aug_loader_d2, val_loader_d2, test_loader_d2 = helpers.create_dataloaders(
    train_d2, augmented_train_d2, val_d2, test_d2, batch_size=32, target=label
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = xrv.models.DenseNet(weights="densenet121-res224-all").to(device)
model.classifier = classes.BaseClassifier(in_features=1024
                                          
                                          )
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
])


In [ ]:
target_label = 'Profusion Label'

print(train_d1.dataset.metadata_df[target_label].value_counts())

In [ ]:
def get_alpha_FLoss(train, target_label):
    """
    Compute a single alpha value for binary focal loss.
    
    """
    class_counts = train.dataset.metadata_df[target_label].value_counts()
    class_counts = class_counts.sort_index()

    # Ensure class counts exist
    if len(class_counts) < 2:
        raise ValueError("Dataset must contain both positive (1) and negative (0) samples.")

    minority = class_counts[0]  # Assuming class 0 is the minority
    majority = class_counts[1]  # Assuming class 1 is the majority

    # Compute alpha as the proportion of the negative class
    alpha = minority / (majority + minority)

    return np.round(alpha, 2)

In [ ]:
print(get_alpha_FLoss(train_d1, 'Profusion Label'))

In [ ]:
def compute_pos_weight(train, target_label):
    """Compute pos_weight for BCEWithLogitsLoss."""
    class_counts = train.dataset.metadata_df[target_label].value_counts()
    class_counts = class_counts.sort_index()

    if len(class_counts) < 2:
        raise ValueError("Dataset must contain both positive (1) and negative (0) samples.")

    N_pos = class_counts[1]  # Positive class count
    N_neg = class_counts[0]  # Negative class count

    pos_weight = torch.tensor([N_neg / N_pos], dtype=torch.float32)  # Must be a tensor

    return pos_weight


In [ ]:
print(compute_pos_weight(train_d1, 'Profusion Label'))

### Higher Resolution Models


In [ ]:
dicom_dir_1 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1'
metadata_1 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1/FileDatabaseWithRadiology.xlsx')
dicom_dir_2 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2'
metadata_2 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2/Database_Training-2024.08.28.xlsx')

d1 = classes.DICOMDataset1(dicom_dir=dicom_dir_1, metadata_df=metadata_1, target_size=224) 
d2 = classes.DICOMDataset2(dicom_dir=dicom_dir_2, metadata_df=metadata_2, target_size=224)

# Split datasets and store indices
train_indices_d1, val_indices_d1, test_indices_d1 = helpers.split_dataset(d1)
train_indices_d2, val_indices_d2, test_indices_d2 = helpers.split_dataset(d2)

# Save indices for later use
split_indices = {
    'd1': {'train': train_indices_d1, 'val': val_indices_d1, 'test': test_indices_d1},
    'd2': {'train': train_indices_d2, 'val': val_indices_d2, 'test': test_indices_d2}
}

label = 'Profusion'
d1.set_target(target_label=label, target_size=512)
d2.set_target(target_label=label, target_size=224)

train_d1 = Subset(d1, train_indices_d1)
val_d1 = Subset(d1, val_indices_d1)
test_d1 = Subset(d1, test_indices_d1)

train_d2 = Subset(d2, train_indices_d2)
val_d2 = Subset(d2, val_indices_d2)
test_d2 = Subset(d2, test_indices_d2)

In [ ]:
model_512 = xrv.models.ResNet(weights="resnet50-res512-all")
model_224 = xrv.models.DenseNet(weights="densenet121-res224-all")


img, label = train_d1[12]
img_2, label_2 = train_d2[12]

print(img.shape, label)
print(img_2.shape, label_2)

In [ ]:
import os

checkpoint_path = 'C:/Users/user-pc/Masters/MSc - Project/prof_and_tb-D2-BCE-OS_best_model_val_kappa.pth'  # Replace with the actual path to your checkpoint file

# Check if the file exists
if os.path.exists(checkpoint_path):
    print(f"Checkpoint file found: {checkpoint_path}")
else:
    print(f"Checkpoint file not found: {checkpoint_path}")

In [ ]:
import torch.optim as optim

str_224 = "densenet121-res224-all"
str_512 = "resnet50-res512-all"

model_x = xrv.models.DenseNet(weights=str_224)
model_x.classifier = classes.BaseClassifier(in_features=1024)


optimizer = optim.Adam(model_x.parameters(), lr=0.001)


# Load the checkpoint
checkpoint_path = 'C:/Users/user-pc/Masters/MSc - Project/prof_and_tb-D2-BCE-OS_best_model_val_kappa.pth'  # Replace with the actual path to your checkpoint file
checkpoint = torch.load(checkpoint_path)

# Load the model state dictionary
model_x.load_state_dict(checkpoint['model_state_dict'])

# Load the optimizer state dictionary
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Access the epoch
epoch = checkpoint['epoch']

print(f"Model and optimizer state loaded from epoch {epoch}")

In [ ]:
augmentations_list = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(15)
])

print(augmentations_list)

img, label = train_d2[12]

# Apply augmentations
augmented_img = augmentations_list(img)

print(augmented_img.shape)

In [ ]:
plt.imshow(augmented_img.squeeze(0), cmap='gray')

In [ ]:
# Define augmentations
augmentations_list = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5)
    transforms.
])

# Create augmented datasets
d1_aug = classes.DICOMDataset1(dicom_dir=dicom_dir_1, metadata_df=metadata_1, transform=augmentations_list)
d2_aug = classes.DICOMDataset2(dicom_dir=dicom_dir_2, metadata_df=metadata_2, transform=augmentations_list)

train_d1 = Subset(d1, train_indices_d1)
train_aug_d1 = Subset(d1_aug, train_indices_d1)

val_d1 = Subset(d1, val_indices_d1)
test_d1 = Subset(d1, test_indices_d1)

train_d2 = Subset(d2, train_indices_d2)
train_aug_d2 = Subset(d2_aug, train_indices_d2)

val_d2 = Subset(d2, val_indices_d2)
test_d2 = Subset(d2, test_indices_d2)

In [ ]:
rand_persp = transforms.RandomPerspective(distortion_scale=0.1, p=1.0, fill=1)

img, label = train_d2[12]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(img.squeeze(0), cmap='gray')
plt.title("Original")

# Apply random perspective
perspective_img = rand_persp(img)
plt.subplot(1, 2, 2)
plt.imshow(perspective_img.squeeze(0), cmap='gray')
plt.title("Random Perspective")

plt.show()


In [ ]:
augs_to_rand_apply = transforms.RandomApply(torch.nn.ModuleList([
    transforms.CenterCrop(np.round(img.shape[1]*0.9).astype(int)),
    transforms.RandomRotation(degrees=(-10,10))
]), p=1)

In [ ]:
trans = augs_to_rand_apply(img)

plt.imshow(trans.squeeze(0), cmap='gray')
plt.show()

In [ ]:
wandb.finish()

In [ ]:
import torch

def salt_and_pepper_noise_tensor(image, prob=0.02):
    """
    Apply salt-and-pepper noise to a PyTorch tensor image.
    
    :param image: PyTorch tensor of shape (C, H, W), values in [0,1].
    :param prob: Probability of a pixel being affected.
    :return: Noisy image tensor.
    """
    assert image.dim() == 3, "Input must be a 3D tensor (C, H, W)"
    
    noisy_image = image.clone()  # Clone to avoid modifying original image
    
    # Generate random noise mask
    rand_tensor = torch.rand_like(image)  # Random values between [0,1]

    # Apply Salt (white pixels)
    noisy_image[rand_tensor < prob / 2] = 1.0  # If image is in [0,1], use 255.0 for [0,255]

    # Apply Pepper (black pixels)
    noisy_image[rand_tensor > 1 - prob / 2] = 0.0

    return noisy_image




In [ ]:
noisy_image = salt_and_pepper_noise_tensor(img, prob=0.1)

plt.imshow(noisy_image.squeeze(0), cmap='gray')
plt.show()


In [ ]:
# Create the transformation pipeline
augs_to_rand_apply = transforms.RandomApply([
    # transforms.CenterCrop(np.round(224 * 0.9).astype(int)),  # Example crop
    transforms.RandomRotation(degrees=(-10, 10)),  
    transforms.Lambda(lambda img: salt_and_pepper_noise_tensor(img, prob=0.05)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05))
], p=1)  # 50% chance of applying the transformations


res = augs_to_rand_apply(img)

plt.imshow(res.squeeze(0), cmap='gray')
plt.show()

In [ ]:
 torch.cuda.memory_summary(device=None, abbreviated=True)

### Resuming training for checkpointed models

In [ ]:
import torch.optim as optim

str_224 = "densenet121-res224-all"
str_512 = "resnet50-res512-all"

model_x = xrv.models.DenseNet(weights=str_224)
model_x.classifier = classes.BaseClassifier(in_features=1024)


optimizer = optim.Adam(model_x.parameters(), lr=0.001)


# Load the checkpoint
checkpoint_path = 'C:/Users/user-pc/Masters/MSc - Project/tb-D2-BCE-OS-aug_p_50_30_final_model.pth'
checkpoint = torch.load(checkpoint_path)

# Load the model state dictionary
model_x.load_state_dict(checkpoint['model_state_dict'])

# Load the optimizer state dictionary
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Access the epoch
epoch = checkpoint['epoch']

print(f"Model and optimizer state loaded from epoch {epoch}")

In [ ]:
wandb.finish()